In [ ]:
import cv2
import os
from matplotlib import pyplot as plt
import numpy as np

# OpenCV: BGR; Matplot: RGB
def show_image(img):
    RGB_im = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(RGB_im)
    plt.show()

In [ ]:
def extract_img(img):
    new_img = img[:, :, 1] # The G part is clearer
    #show_image(new_img)
    return new_img

## Save the Image
Called by split_img()

In [ ]:
def save_img(img, filename):
    name = filename + ".jpg"
    cv2.imwrite(name, img)
    return name

## Split the image into 4 digit image
### Parameters
- img: return from cv2.imread
- path: The directory for putting cutted image
- filename: The image file name

### Return
An array contains saved cutted image name

In [ ]:
def split_img(img, dir_path, filename):
    saved_file_name = []

    if img is None:
        return

    rows = img.shape[1]

    step = rows//4
    # if step is too small which means the picture is too small, skip it
    if (step < 100):
        return
    
    name = os.path.splitext(filename)
    
    old_row = 0
    for i in range(1, 5):
        small_img = img[:, old_row:step*i]
        #show_image(small_img)
        old_row = step*i
        
        # save cutted image
        img_name = name[0] + "-" + str(i)
        full_path = os.path.join(dir_path, img_name)
        small_img_name = save_img(small_img, full_path)
        # store the cutted image file name
        saved_file_name.append(small_img_name)

    return saved_file_name

In [ ]:
DIR = 'test_images/'

for root, subdirs, files in os.walk(DIR):
    print("root:", root)
    for filename in files:
        file_path = os.path.join(root, filename)
        if file_path.endswith(".jpg"):
            print("Process ", file_path)
            img = cv2.imread(file_path, cv2.IMREAD_COLOR)
            #show_image(img)
            G_img = extract_img(img)
            
            # stored into root (test_images)
            cut_img_path = split_img(G_img, root, filename)
            print(cut_img_path)
print("Done")